# Guía de entornos virtuales (CMD)


# Extra: Gestionar **varios entornos** en VS Code

Cuando trabajas con múltiples proyectos, crea **un entorno por proyecto** para evitar conflictos de versiones.

## 1) Crear un entorno por proyecto (CMD)
En cada carpeta de proyecto ejecuta:

```cmd
python -m venv .venv
.venv\Scripts\activate.bat
```
Opcional: usa nombres descriptivos si manejas muchos entornos:
```cmd
python -m venv .venv_django
python -m venv .venv_flask
```

## 2) Seleccionar el intérprete correcto en VS Code
Con la carpeta del proyecto abierta:
1. `Ctrl+Shift+P` → **Python: Select Interpreter**.
2. Elige el que apunte a `.<entorno>\Scripts\python.exe`.
3. VS Code guardará la selección en `.vscode/settings.json` de ese proyecto.

## 3) Cambiar entre proyectos
- Al abrir **proyecto1**, verás algo como `Python 3.x ('.venv': venv)` abajo a la derecha.
- Al abrir **proyecto2**, se actualizará automáticamente al `.venv` de ese proyecto.

## 4) Verificar el entorno activo
En la **terminal integrada** debería verse el prefijo del entorno:
```
(.venv) C:\Users\TuUsuario\Desktop\proyecto1>
```
Y en la barra de estado (abajo derecha):
`Python 3.x ('.venv': venv)` o el nombre que le hayas dado (p. ej. `.venv_django`).

## 5) Buenas prácticas
- Mantén un `requirements.txt` por proyecto:
  ```cmd
  pip freeze > requirements.txt
  ```
  Para recrear el entorno en otra máquina:
  ```cmd
  pip install -r requirements.txt
  ```
- No mezcles dependencias de proyectos: instala siempre dentro del entorno correspondiente.
- Si cambias mucho de entorno, considera nombres distintivos para reconocerlos rápido (`.venv_api`, `.venv_data`, etc.).

## 6) Trucos útiles
- **Forzar selección manual** si el `.venv` no aparece:
  `Ctrl+Shift+P` → *Python: Select Interpreter* → *Enter Interpreter Path* → *Find...* → selecciona `.<entorno>\Scripts\python.exe`.
- **Reiniciar analizador / terminal** si no reconoce paquetes:
  `Ctrl+Shift+P` → *Python: Restart Language Server* o *Developer: Reload Window*.
  Cierra y reabre la terminal integrada (`Ctrl+ñ`).


# Extra: `requirements.txt` y recrear entornos desde cero (CMD)

## 1) Crear `requirements.txt` del proyecto actual
Con tu entorno activo `(.venv)` ejecuta en **CMD**:
```cmd
pip freeze > requirements.txt
```
Esto genera un listado **exacto** de versiones usadas por tu proyecto.

## 2) Instalar desde `requirements.txt` en un entorno nuevo
En una carpeta vacía o en el mismo proyecto (tras crear un `.venv` nuevo):
```cmd
python -m venv .venv
.venv\Scripts\activate.bat
python -m pip install --upgrade pip
pip install -r requirements.txt
```

## 3) Recomendaciones de versionado
- **Producción/reproducibilidad:** usa versiones fijas (p. ej. `flask==3.0.3`).
- **Desarrollo flexible:** puedes permitir rangos (`flask>=3.0,<4`).
- Antes de publicar el `requirements.txt`, considera **limpiarlo** (quitar paquetes que no necesitas).

## 4) Script CMD para recrear el entorno automáticamente
Crea un archivo `setup_venv_from_requirements.cmd` y pégale esto:
```cmd
@echo off
setlocal enableextensions enabledelayedexpansion
REM Ruta del entorno (por defecto .venv)
set VENV_DIR=.venv

if not exist "%VENV_DIR%" (
  echo [*] Creando entorno virtual en %VENV_DIR%
  python -m venv %VENV_DIR%
)

echo [*] Activando entorno
call %VENV_DIR%\Scripts\activate.bat

echo [*] Actualizando pip
python -m pip install --upgrade pip

if exist requirements.txt (
  echo [*] Instalando dependencias desde requirements.txt
  pip install -r requirements.txt
) else (
  echo [!] No se encontro requirements.txt en el directorio actual
)

echo [*] Listo. Entorno activo en %VENV_DIR%
python --version
where python
pip list
endlocal
```
Guárdalo en la **raíz de tu proyecto** y ejecútalo con doble clic o desde CMD:
```cmd
setup_venv_from_requirements.cmd
```

## 5) Plantilla básica de `requirements.txt`
Ejemplo mínimo (ajústalo a tu proyecto):
```
flask==3.0.3
pandas==2.2.3
reportlab==4.4.4
pillow==11.3.0
tzdata==2025.2
```

## 6) Exportar solo lo necesario
Si `pip freeze` genera demasiadas dependencias, puedes **escribir manualmente** un `requirements.in` con tus paquetes principales y luego usar herramientas como `pip-tools` (opcional) para compilarlo a `requirements.txt`. Para mantenerlo simple, puedes empezar editándolo a mano.


---
# ANEXO: Guía completa de scripts `.cmd` para gestionar entornos
A continuación se incluye, íntegramente, la guía detallada para usar los 4 scripts `.cmd` (`setup_venv_from_requirements.cmd`, `reset_venv_from_requirements.cmd`, `provision_and_open_venv.cmd`, `open_venv_here.cmd`).


# Gestión completa de entornos virtuales con **scripts .cmd** (Windows / CMD)

Este cuaderno explica, paso a paso, **cómo usar cuatro scripts .cmd** para crear, resetear, provisionar y abrir entornos virtuales de Python en Windows (CMD).

> **Requisitos previos**
> - Tener **Python 3.x** instalado y agregado al **PATH** (que `python --version` funcione en CMD).
> - Trabajar en **CMD (Símbolo del sistema)**, no PowerShell.
> - Coloca los scripts en la **raíz** de tu proyecto (junto a `.venv` y `requirements.txt`).


## Estructura recomendada del proyecto
```
mi_proyecto/
 ├─ .venv/                 (carpeta del entorno virtual; puede no existir aún)
 ├─ requirements.txt       (opcional, recomendado)
 ├─ setup_venv_from_requirements.cmd
 ├─ reset_venv_from_requirements.cmd
 ├─ provision_and_open_venv.cmd
 └─ open_venv_here.cmd
```


## Conceptos clave (rápido)
- **Entorno virtual**: carpeta aislada con las librerías del proyecto.
- **Activar el entorno**: hace que `python` y `pip` apunten al Python de `.venv`.
- **`requirements.txt`**: lista las dependencias del proyecto.
- **CMD vs PowerShell**: aquí usamos **CMD**. Si VS Code abre PowerShell, cambia el perfil a **Command Prompt (cmd)**.


# 1) `setup_venv_from_requirements.cmd` — Provisionar el entorno desde `requirements.txt`

### ¿Qué hace?
- Crea `.venv` si no existe.
- Activa **temporalmente** el entorno dentro del script.
- Actualiza `pip`.
- Instala dependencias desde `requirements.txt` (si existe).
- Muestra versión/ruta de Python y paquetes.
- **No** deja el entorno activado al terminar.

### ¿Cuándo usarlo?
- Primera vez en una PC.
- Cuando recibes/actualizas `requirements.txt` y quieres instalar todo.

### Cómo usarlo (CMD)


```cmd
cd C:\\Users\\TU\\Desktop\\mi_proyecto
setup_venv_from_requirements.cmd

REM Al terminar, activa el entorno para trabajar:
.venv\\Scripts\\activate.bat
```


# 2) `reset_venv_from_requirements.cmd` — Borrar y recrear el entorno (con confirmación)

### ¿Qué hace?
- Pregunta si quieres **borrar y recrear** `.venv`.
- Si confirmas:
  - Elimina `.venv`.
  - Crea uno nuevo.
  - Activa, actualiza `pip` e instala desde `requirements.txt` (si existe).
  - Deja la ventana abierta con el entorno **activo**.

### ¿Cuándo usarlo?
- Si el entorno está roto o con conflictos.
- Si quieres empezar **desde cero**.

### Cómo usarlo (CMD)
```cmd
cd C:\\Users\\TU\\Desktop\\mi_proyecto
reset_venv_from_requirements.cmd
```


# 3) `provision_and_open_venv.cmd` — Provisionar y **quedarte** con `.venv` activo

### ¿Qué hace?
- Crea `.venv` si no existe.
- Activa el entorno.
- Actualiza `pip`.
- Instala desde `requirements.txt` si existe.
- **Deja la terminal abierta** con el entorno activo.

### ¿Cuándo usarlo?
- Quieres un **todo-en-uno** para empezar a trabajar ya.

### Cómo usarlo (CMD)
```cmd
cd C:\\Users\\TU\\Desktop\\mi_proyecto
provision_and_open_venv.cmd
```


# 4) `open_venv_here.cmd` — Abrir CMD con `.venv` **ya activado**

### ¿Qué hace?
- No crea ni instala nada.
- Abre una CMD con el entorno **activado** (si `.venv` existe).

### ¿Cuándo usarlo?
- Ya tienes `.venv` y quieres **abrirlo rápido** para trabajar.

### Cómo usarlo
- Doble clic sobre `open_venv_here.cmd`, **o**:
```cmd
cd C:\\Users\\TU\\Desktop\\mi_proyecto
open_venv_here.cmd
```


## Verificación (después de cualquier script)
En CMD con `.venv` activo deberías ver el prefijo `(.venv)` y estos comandos deben responder correctamente:
```cmd
python --version
where python
pip list
```
En un **notebook** dentro de VS Code, prueba:
```python
import sys
sys.executable
```
Debería apuntar a `...\\.venv\\Scripts\\python.exe`.


## Solución de problemas (FAQ)

**`'python' no se reconoce`**
- Reinstala Python marcando **Add Python to PATH**. Abre un CMD nuevo.

**No aparece `.venv` en VS Code o no usa mi entorno**
- `Ctrl+Shift+P` → *Python: Select Interpreter* → selecciona `...\\.venv\\Scripts\\python.exe`.
- `Ctrl+Shift+P` → *Python: Restart Language Server* o *Developer: Reload Window*.
- Cierra y vuelve a abrir la **terminal integrada** (`Ctrl+ñ`).

**Error al borrar `.venv` en el script de reset**
- Cierra VS Code y cualquier terminal que esté usando `.venv`.
- Intenta de nuevo; si persiste, reinicia Windows.

**No tengo `requirements.txt`**
- Puedes generarlo con:
```cmd
pip freeze > requirements.txt
```
o escribirlo a mano con los paquetes principales.

**Sigue sin reconocer paquetes en VS Code**
- Verifica en la barra de estado: `Python 3.x ('.venv': venv)`.
- Asegúrate de haber abierto la **carpeta** del proyecto (no solo archivos sueltos).


## Extras útiles
- **Crear `requirements.txt` del entorno actual**:
```cmd
pip freeze > requirements.txt
```
- **Instalar desde `requirements.txt`** en un entorno nuevo:
```cmd
python -m venv .venv
.venv\\Scripts\\activate.bat
python -m pip install --upgrade pip
pip install -r requirements.txt
```
- **Anclar CMD al entorno** (alternativa manual):
```cmd
.venv\\Scripts\\activate.bat
```
